# How does availability of wifi affect hospital ratings?

In [3]:
## import libraries
import requests

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
import geocoder # import geocoder to get latitude and longitude
import json # library to handle JSON files

import geopy.geocoders
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geopy.geocoders.options.default_timeout = 60 # otherwise it will timeout at some point

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
## Define foursquare credentials and version (date)
CLIENT_ID = 'U1ALKNVSQ5OT5GCDQTE3NTUSVQVANLLFPZBOW3BMYR53AG20' # your Foursquare ID
CLIENT_SECRET = '2BGUAXJ14CEJNJQHSWRLNGA5IOJCMBSOAENHD5INTJOHLP45' # your Foursquare Secret
VERSION = '20191111'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U1ALKNVSQ5OT5GCDQTE3NTUSVQVANLLFPZBOW3BMYR53AG20
CLIENT_SECRET:2BGUAXJ14CEJNJQHSWRLNGA5IOJCMBSOAENHD5INTJOHLP45


In [6]:
# Scrape hospital names, cities and basic stats in New Jersey from website
url_nj_hosp = 'https://www.ahd.com/states/hospital_NJ.html'
html = requests.get(url_nj_hosp).content
df_list = pd.read_html(html)
hosp_nj = df_list[1]
hosp_nj.to_csv('NJ_hospital_data_ahd.csv')
hosp_nj.head()

,Hospital Name,City,StaffedBeds,TotalDischarges,PatientDays,Gross PatientRevenue ($000)
0,Palisades Medical Center,North Bergen,202,9575,41821,"$955,944"
1,AtlantiCare Regional Medical Center - Mainland...,Pomona,0,0,0,$0
2,AtlantiCare Regional Medical Center -Atlantic ...,Atlantic City,520,29917,129930,"$3,608,586"
3,Bayonne Medical Center,Bayonne,163,5083,23070,"$1,603,840"
4,Bayshore Medical Center,Holmdel,169,7212,37434,"$853,103"


In [7]:
# Scrape hospital names, cities and basic stats in New York (state) from website
url_ny_hosp = 'https://www.ahd.com/states/hospital_NY.html'
html = requests.get(url_ny_hosp).content
df_list = pd.read_html(html)
hosp_ny = df_list[1]
hosp_ny.to_csv('NY_hospital_data_ahd.csv')
hosp_ny.head()

,Hospital Name,City,StaffedBeds,TotalDischarges,PatientDays,Gross PatientRevenue ($000)
0,Long Island Community Hospital,Patchogue,235,11756,69960,"$1,419,960"
1,SUNY Downstate Medical Center University Hospi...,Brooklyn,323,10599,68972,"$997,255"
2,A.O. Fox Hospital,Oneonta,191,2168,8222,"$174,018"
3,Adirondack Medical Center at Saranac Lake,Saranac Lake,155,2001,7868,"$264,858"
4,Albany Medical Center,Albany,793,39130,214165,"$3,152,876"


In [8]:
## Rename columns for consistency, add state column and combine state data frames
hosp_nj.rename(columns={"Hospital Name": "name", 
                        "City": "city", 
                        "StaffedBeds": "staffed_beds", 
                        "TotalDischarges": "total_discharges", 
                        "PatientDays": "patient_days",
                        "Gross PatientRevenue ($000)": "gross_patient_revenue"}, inplace = True)
hosp_nj["state"] = "nj"
hosp_ny.rename(columns={"Hospital Name": "name", 
                        "City": "city", 
                        "StaffedBeds": "staffed_beds", 
                        "TotalDischarges": "total_discharges", 
                        "PatientDays": "patient_days",
                        "Gross PatientRevenue ($000)": "gross_patient_revenue"}, inplace = True)
hosp_ny["state"] = "ny"
hosp_states = pd.concat([hosp_nj, hosp_ny], ignore_index = True)
hosp_states.head()

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state
0,Palisades Medical Center,North Bergen,202,9575,41821,"$955,944",nj
1,AtlantiCare Regional Medical Center - Mainland...,Pomona,0,0,0,$0,nj
2,AtlantiCare Regional Medical Center -Atlantic ...,Atlantic City,520,29917,129930,"$3,608,586",nj
3,Bayonne Medical Center,Bayonne,163,5083,23070,"$1,603,840",nj
4,Bayshore Medical Center,Holmdel,169,7212,37434,"$853,103",nj


In [9]:
## check for missing data
hosp_states.isnull().sum()

name                     0
city                     2
staffed_beds             0
total_discharges         0
patient_days             0
gross_patient_revenue    0
state                    0
dtype: int64

In [10]:
hosp_states.loc[hosp_states.city.isnull()]

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state
75,T O T A L,NaN,20210,876924,4312872,"$123,883,550",nj
265,T O T A L,NaN,56744,2193906,11748920,"$245,506,269",ny


In [11]:
## drop total rows from hospital data and reset index to be continous from 1, 2, ..., len(hosp_states)
hosp_states = hosp_states.dropna()
hosp_states = hosp_states.reset_index(drop=True)
hosp_states.iloc[73:78,]

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state
73,Virtua Voorhees Hospital,Voorhees,587,35329,144883,"$3,844,933",nj
74,Virtua Willingboro Hospital,Willingboro,144,5975,30130,"$793,196",nj
75,Long Island Community Hospital,Patchogue,235,11756,69960,"$1,419,960",ny
76,SUNY Downstate Medical Center University Hospi...,Brooklyn,323,10599,68972,"$997,255",ny
77,A.O. Fox Hospital,Oneonta,191,2168,8222,"$174,018",ny


In [12]:
# add address column, which we can use to search for geographical coordinates of cities (bc cities with similar names might exist elsewhere...)
hosp_states["address"] = hosp_states.city + ", " + hosp_states.state
hosp_states.head()

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address
0,Palisades Medical Center,North Bergen,202,9575,41821,"$955,944",nj,"North Bergen, nj"
1,AtlantiCare Regional Medical Center - Mainland...,Pomona,0,0,0,$0,nj,"Pomona, nj"
2,AtlantiCare Regional Medical Center -Atlantic ...,Atlantic City,520,29917,129930,"$3,608,586",nj,"Atlantic City, nj"
3,Bayonne Medical Center,Bayonne,163,5083,23070,"$1,603,840",nj,"Bayonne, nj"
4,Bayshore Medical Center,Holmdel,169,7212,37434,"$853,103",nj,"Holmdel, nj"


In [13]:
addresses = hosp_states.address.unique()
addresses[0]

'North Bergen, nj'

In [14]:
## set geocoder timeout quite high, to avoid errors
geopy.geocoders.options.default_timeout = 60 

In [16]:
len(addresses)

179

In [ ]:
## get latitude and longitude for each city in the hosp_states dataframe
latitudes = [None] * len(addresses)
longitudes = [None] * len(addresses)
lost_cities = list()
for i in range(len(addresses)):
    address = addresses[i]
    geolocator = Nominatim(user_agent="random_app_name")
    loc = geolocator.geocode(address)
    if loc:
        lat = loc.latitude
        lng = loc.longitude
        print('The geograpical coordinates of {} are {}, {}.'.format(address, lat, lng))
        latitudes[i] = lat
        longitudes[i] = lng
    else:
        latitudes[i] = float('NaN')
        longitudes[i] = float('NaN')
        lost_cities.append(address)
        print('The geograpical coordinates of {} are not available.'.format(address))

In [19]:
## any place we could not find the coordinates for?
lost_cities

[]

In [20]:
# create addresses data frame
df_addresses = pd.DataFrame({'address': addresses, 'lat': latitudes, 'lng': longitudes})
df_addresses.head()

,address,lat,lng
0,"North Bergen, nj",40.804267,-74.012084
1,"Pomona, nj",39.464194,-74.545266
2,"Atlantic City, nj",39.364285,-74.422935
3,"Bayonne, nj",40.668714,-74.114309
4,"Holmdel, nj",40.345109,-74.184032


In [21]:
hosp_states.head()

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address
0,Palisades Medical Center,North Bergen,202,9575,41821,"$955,944",nj,"North Bergen, nj"
1,AtlantiCare Regional Medical Center - Mainland...,Pomona,0,0,0,$0,nj,"Pomona, nj"
2,AtlantiCare Regional Medical Center -Atlantic ...,Atlantic City,520,29917,129930,"$3,608,586",nj,"Atlantic City, nj"
3,Bayonne Medical Center,Bayonne,163,5083,23070,"$1,603,840",nj,"Bayonne, nj"
4,Bayshore Medical Center,Holmdel,169,7212,37434,"$853,103",nj,"Holmdel, nj"


In [22]:
## Finally, combine city data back with state hospital data
merged_left = pd.merge(left=hosp_states,right=df_addresses, how='left', left_on='address', right_on='address')
merged_left.loc[merged_left.city == "Buffalo"] # check if left join did what I wanted

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address,lat,lng
93,Buffalo General Medical Center,Buffalo,1068,50926,264758,"$3,326,172",ny,"Buffalo, ny",42.886717,-78.878392
94,Buffalo VA Medical Center,Buffalo,0,0,0,$0,ny,"Buffalo, ny",42.886717,-78.878392
107,Erie County Medical Center,Buffalo,906,16670,102375,"$1,025,323",ny,"Buffalo, ny",42.886717,-78.878392
141,Mercy Hospital of Buffalo,Buffalo,470,18591,91807,"$1,036,453",ny,"Buffalo, ny",42.886717,-78.878392
208,Roswell Park Cancer Institute,Buffalo,133,5307,40867,"$1,622,622",ny,"Buffalo, ny",42.886717,-78.878392
227,Sisters of Charity Hospital,Buffalo,466,15135,78523,"$753,650",ny,"Buffalo, ny",42.886717,-78.878392
228,Sisters of Charity Hospital - Saint Joseph Campus,Buffalo,119,6318,36997,"$189,421",ny,"Buffalo, ny",42.886717,-78.878392


In [23]:
## update hosp_states data
hosp_states = merged_left

In [24]:
hosp_states.head(4)

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address,lat,lng
0,Palisades Medical Center,North Bergen,202,9575,41821,"$955,944",nj,"North Bergen, nj",40.804267,-74.012084
1,AtlantiCare Regional Medical Center - Mainland...,Pomona,0,0,0,$0,nj,"Pomona, nj",39.464194,-74.545266
2,AtlantiCare Regional Medical Center -Atlantic ...,Atlantic City,520,29917,129930,"$3,608,586",nj,"Atlantic City, nj",39.364285,-74.422935
3,Bayonne Medical Center,Bayonne,163,5083,23070,"$1,603,840",nj,"Bayonne, nj",40.668714,-74.114309


In [25]:
start_lat = hosp_states.loc[hosp_states.city == "Johnson City", "lat"] # Start at a city roughly in the middle
start_lng = hosp_states.loc[hosp_states.city == "Johnson City", "lng"] 
print(start_lat)
print(start_lng)

248    42.115631
Name: lat, dtype: float64
248   -75.958809
Name: lng, dtype: float64


In [26]:
# create map with new york state and new jersey cities / areas
start_lat = 42.115631 # Start at a city roughly in the middle
start_lng = -75.958809 
map_ny_nj_hospitals = folium.Map(location=[start_lat, start_lng], zoom_start=7)

# add markers to map
for lat, lng, city in zip(hosp_states['lat'], hosp_states['lng'], hosp_states['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny_nj_hospitals)  
    
map_ny_nj_hospitals

In [ ]:
## Loop through addresses in NJ and NY and find adjecent hospitals using the Foursquare API
fs_hospital_list = list()
radius = 25000 # hospitals within 15000 meters
for i in range(len(df_addresses)):
    
    # create the API request URL
    lat = df_addresses.lat[i]
    lng = df_addresses.lng[i]
    search_query = "hospital"
    LIMIT = 50
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        search_query,
        lat, 
        lng, 
        radius, 
        LIMIT)

    # make the GET request
    results = requests.get(url).json()

    # put json results into dataframe
    hospitals = results['response']['groups'][0]['items']
    nearby_hospitals = json_normalize(hospitals)
    
    # accumulate dataframes from each foursquare API call in a list
    fs_hospital_list.append(nearby_hospitals)
    
    print('Requesting foursquare hospital information for {}.'.format(df_addresses.address.loc[i]))

In [35]:
## combine data frames from individual API calls into one large data frame
df_fs_hospitals = pd.concat(fs_hospital_list, sort = False)
df_fs_hospitals.describe()

,reasons.count,venue.location.distance,venue.location.lat,venue.location.lng,venue.photos.count
count,6595.0,6595.000000,6595.000000,6595.000000,6595.0
mean,0.0,10789.513723,40.998318,-74.469971,0.0
std,0.0,6541.276625,0.936465,1.250814,0.0
min,0.0,81.000000,38.975600,-79.569922,0.0
25%,0.0,5292.000000,40.585134,-74.606796,0.0
50%,0.0,10435.000000,40.780781,-74.056677,0.0
75%,0.0,15559.000000,41.028751,-73.840528,0.0
max,0.0,25035.000000,45.030072,-72.316806,0.0


In [36]:
## extract columns of interest
filtered_columns = ['venue.id', 'venue.location.city', 'venue.location.lat', 'venue.location.lng', 'venue.location.state', 'venue.name']
df_fs_hospitals = df_fs_hospitals.loc[:, filtered_columns]
df_fs_hospitals.head()

,venue.id,venue.location.city,venue.location.lat,venue.location.lng,venue.location.state,venue.name
0,4a4cc69df964a52040ad1fe3,New York,40.773796,-73.961115,NY,Lenox Hill Hospital
1,4a78348ff964a52046e51fe3,New York,40.765348,-73.952689,NY,Hospital for Special Surgery
2,4b06ab0bf964a5206dee22e3,Hackensack,40.884080,-74.056677,NJ,Hackensack University Medical Center
3,4bbe05aca8cf76b0b07fb2fd,New York,40.841372,-73.942563,NY,Milstein Hospital Building
4,4b9e3d22f964a5203bd436e3,Teaneck,40.883524,-74.010959,NJ,Holy Name Medical Center


In [37]:
## delete redundant hospitals
df_fs_hospitals.drop_duplicates(subset='venue.id', keep="last", inplace = True)
#df_fs_hospitals["venue.id"].value_counts()
len(df_fs_hospitals)

2112

In [38]:
## Rename columns in foursquare hospital data frame
df_fs_hospitals.rename(columns={"venue.id": "fs_id",
                                      "venue.location.city": "fs_city",
                                      "venue.location.lat": "fs_lat",
                                      "venue.location.lng": "fs_lng",
                                      "venue.location.state": "fs_state",
                                      "venue.name": "fs_name"}, inplace = True)
df_fs_hospitals.head()

,fs_id,fs_city,fs_lat,fs_lng,fs_state,fs_name
32,52f91afe11d2af499982e107,New York,40.805388,-73.961665,NY,St. Luke's Cafeteria
33,4af2b0def964a5203fe821e3,New York,40.805570,-73.961321,NY,Mount Sinai St. Luke's
39,4ea643524690ee4468d62704,New York,40.789614,-73.954518,NY,Mount Sinai Labor & Delivery
40,4a6fe770f964a5201bd71fe3,New York,40.790095,-73.953917,NY,Kravis Children's Hospital
41,4ca389bd1ee76dcba527fadd,New York,40.790021,-73.953409,NY,Annenberg Building


In [39]:
## check state names, adjust to fit with list from american hospital directory and kick out hospitals in adjacent states
df_fs_hospitals['fs_state'] = df_fs_hospitals['fs_state'].str.lower()
df_fs_hospitals['fs_state'].head()

32    ny
33    ny
39    ny
40    ny
41    ny
Name: fs_state, dtype: object

In [41]:
## what labels do we have for state?
df_fs_hospitals['fs_state'].unique()

In [42]:
## substitute spelled-out new jersey or new york for abbreviations
df_fs_hospitals.loc[df_fs_hospitals['fs_state'] == 'new jersey', 'fs_state'] = 'nj'
df_fs_hospitals.loc[df_fs_hospitals['fs_state'] == 'new jersey', 'fs_state'] = 'nj'
## and kick out other states
df_fs_hospitals = df_fs_hospitals[~df_fs_hospitals['fs_state'].isin(['pa', 'pennsylvania', 'ca', 'on', 'ct', 'connecticut', 'de', 'vt'])]
## drop nans
df_fs_hospitals.dropna(inplace = True)
## reset index
df_fs_hospitals.reset_index(drop=True, inplace = True)
len(df_fs_hospitals)

1863

In [43]:
## we should now only have ny and nj as values for fs_state
df_fs_hospitals['fs_state'].unique()

array(['ny', 'nj'], dtype=object)

In [44]:
## Add address column to foursquare data to merge with american hospital association data
df_fs_hospitals['fs_address'] = df_fs_hospitals['fs_city'] + ", " + df_fs_hospitals['fs_state']
df_fs_hospitals.head()

,fs_id,fs_city,fs_lat,fs_lng,fs_state,fs_name,fs_address
0,52f91afe11d2af499982e107,New York,40.805388,-73.961665,ny,St. Luke's Cafeteria,"New York, ny"
1,4af2b0def964a5203fe821e3,New York,40.805570,-73.961321,ny,Mount Sinai St. Luke's,"New York, ny"
2,4ea643524690ee4468d62704,New York,40.789614,-73.954518,ny,Mount Sinai Labor & Delivery,"New York, ny"
3,4a6fe770f964a5201bd71fe3,New York,40.790095,-73.953917,ny,Kravis Children's Hospital,"New York, ny"
4,4ca389bd1ee76dcba527fadd,New York,40.790021,-73.953409,ny,Annenberg Building,"New York, ny"


In [45]:
## It's probably safer to convert all strings to lowercase
df_fs_hospitals['fs_address'] = df_fs_hospitals['fs_address'].str.lower()
df_fs_hospitals['fs_name'] = df_fs_hospitals['fs_name'].str.lower()
hosp_states['name'] = hosp_states['name'].str.lower()

In [47]:
hosp_states.head()

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address,lat,lng
0,palisades medical center,North Bergen,202,9575,41821,"$955,944",nj,"North Bergen, nj",40.804267,-74.012084
1,atlanticare regional medical center - mainland...,Pomona,0,0,0,$0,nj,"Pomona, nj",39.464194,-74.545266
2,atlanticare regional medical center -atlantic ...,Atlantic City,520,29917,129930,"$3,608,586",nj,"Atlantic City, nj",39.364285,-74.422935
3,bayonne medical center,Bayonne,163,5083,23070,"$1,603,840",nj,"Bayonne, nj",40.668714,-74.114309
4,bayshore medical center,Holmdel,169,7212,37434,"$853,103",nj,"Holmdel, nj",40.345109,-74.184032


In [48]:
df_fs_hospitals.head()

,fs_id,fs_city,fs_lat,fs_lng,fs_state,fs_name,fs_address
0,52f91afe11d2af499982e107,New York,40.805388,-73.961665,ny,st. luke's cafeteria,"New York, ny"
1,4af2b0def964a5203fe821e3,New York,40.805570,-73.961321,ny,mount sinai st. luke's,"New York, ny"
2,4ea643524690ee4468d62704,New York,40.789614,-73.954518,ny,mount sinai labor & delivery,"New York, ny"
3,4a6fe770f964a5201bd71fe3,New York,40.790095,-73.953917,ny,kravis children's hospital,"New York, ny"
4,4ca389bd1ee76dcba527fadd,New York,40.790021,-73.953409,ny,annenberg building,"New York, ny"


In [46]:
## Finally, merge foursquare hospital list with american hospital association list (will drop the number substantially)
hosp_merged = pd.merge(left=hosp_states,right=df_fs_hospitals, how='left', left_on='name', right_on='fs_name')
hosp_merged.loc[hosp_merged.address == "buffalo, ny"] # check if left join did what I wanted

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address,lat,lng,fs_id,fs_city,fs_lat,fs_lng,fs_state,fs_name,fs_address


In [470]:
df_fs_hospitals.dtypes

fs_id          object
fs_city        object
fs_lat        float64
fs_lng        float64
fs_state       object
fs_name        object
fs_address     object
dtype: object

In [471]:
hosp_states.dtypes

name                      object
city                      object
staffed_beds               int64
total_discharges           int64
patient_days               int64
gross_patient_revenue     object
state                     object
address                   object
lat                      float64
lng                      float64
dtype: object

In [472]:
# Function to find all close matches of  
# input string in given list of possible strings 
from difflib import get_close_matches 
  
print(get_close_matches(hosp_states.loc[0,'name'], df_fs_hospitals.loc[:,'fs_name'])) 


['salem medical center', 'maimonides medical center', 'alice hyde medical center']


In [479]:
hosp_states.loc[0,'city']

'North Bergen'

In [481]:
print(get_close_matches(hosp_states.loc[0,'city'], df_fs_hospitals.loc[:,'fs_city']))

TypeError: object of type 'float' has no len()

In [414]:
venue_id = df_fs_hospitals.loc[1,'fs_id']
venue_id

'4af2b0def964a5203fe821e3'

In [425]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url    

'https://api.foursquare.com/v2/venues/4af2b0def964a5203fe821e3?client_id=U1ALKNVSQ5OT5GCDQTE3NTUSVQVANLLFPZBOW3BMYR53AG20&client_secret=2BGUAXJ14CEJNJQHSWRLNGA5IOJCMBSOAENHD5INTJOHLP45&v=20191111'

In [426]:
result = requests.get(url).json()
result['response']['venue']['likes']['count']

26

In [417]:
result['response']['venue']

{'id': '4af2b0def964a5203fe821e3',
 'name': "Mount Sinai St. Luke's",
 'contact': {'phone': '2125234000',
  'formattedPhone': '(212) 523-4000',
  'twitter': 'mountsinainyc',
  'facebook': '190647927632800',
  'facebookUsername': 'mountsinainyc',
  'facebookName': 'The Mount Sinai Hospital'},
 'location': {'address': '1111 Amsterdam Avenue',
  'crossStreet': 'at West 114th St.',
  'lat': 40.80557,
  'lng': -73.961321,
  'labeledLatLngs': [{'label': 'display', 'lat': 40.80557, 'lng': -73.961321}],
  'postalCode': '10025',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['1111 Amsterdam Avenue (at West 114th St.)',
   'New York, NY 10025',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/mount-sinai-st-lukes/4af2b0def964a5203fe821e3',
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categ

In [418]:
# put json results into dataframe
test = results['response']['groups'][0]['items']
#nearby_hospitals = json_normalize(hospitals)

In [419]:
result = requests.get(url).json()
print(result['response']['venue'].keys())


dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'venueRatingBlacklisted', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


In [448]:
result['response']['venues']['tips']['phrases']

KeyError: 'venues'

In [459]:
result['response']['venue']['tips']['groups']

[{'type': 'others',
  'name': 'All tips',
  'count': 26,
  'items': [{'id': '4d756820381fa35da227275d',
    'createdAt': 1299540000,
    'text': 'Best ER service',
    'type': 'user',
    'canonicalUrl': 'https://foursquare.com/item/4d756820381fa35da227275d',
    'lang': 'en',
    'likes': {'count': 6,
     'groups': [{'type': 'others',
       'count': 6,
       'items': [{'id': '44774553',
         'firstName': 'Jonathan',
         'lastName': 'Ramos',
         'gender': 'male',
         'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
          'suffix': '/44774553-ZG2Q5YOSUJFLAOGC.jpg'}},
        {'id': '8852447',
         'firstName': 'Santi',
         'gender': 'male',
         'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
          'suffix': '/8852447-3CQLDNH1SZVYIBWJ.jpg'}}]}],
     'summary': '6 likes'},
    'logView': True,
    'agreeCount': 2,
    'disagreeCount': 0,
    'todo': {'count': 0},
    'user': {'id': '6133945',
     'firstName': 'Jenna',
     'ge

In [ ]:
## Not so sure if it actually makes sense to combine them anymore... simply continue only with the FS data
## What we need to know now is whether there is wifi access or not
for venue_id in df_fs_hospitals['fs_id']
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    

In [336]:
## Now sub-select the hospitals from the original list (from the american hospital directory)
hosp_states.head()

,name,city,staffed_beds,total_discharges,patient_days,gross_patient_revenue,state,address,lat,lng
0,Palisades Medical Center,North Bergen,202,9575,41821,"$955,944",nj,"North Bergen, nj",40.804267,-74.012084
1,AtlantiCare Regional Medical Center - Mainland...,Pomona,0,0,0,$0,nj,"Pomona, nj",39.464194,-74.545266
2,AtlantiCare Regional Medical Center -Atlantic ...,Atlantic City,520,29917,129930,"$3,608,586",nj,"Atlantic City, nj",39.364285,-74.422935
3,Bayonne Medical Center,Bayonne,163,5083,23070,"$1,603,840",nj,"Bayonne, nj",40.668714,-74.114309
4,Bayshore Medical Center,Holmdel,169,7212,37434,"$853,103",nj,"Holmdel, nj",40.345109,-74.184032


In [78]:
len(df_filt)

50

In [2]:
tips = result['response']['tips']['items']

tip = result['response']['tips']['items'][0]
tip.keys()

NameError: name 'result' is not defined